<a href="https://colab.research.google.com/github/kaaath-i/emotion-classification-bert-models/blob/main/Final_Project_ZEILNHOFER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**VU Introduction to Computational Linguistics**

Katharina Zeilnhofer

## **Final Project**


**Task**: Sentiment Analysis - Emotion Classification

*Dataset* : https://huggingface.co/datasets/mteb/emotion

*Model 1* : https://huggingface.co/distilbert/distilbert-base-cased

*Model 2* : https://huggingface.co/FacebookAI/roberta-base

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install bertviz transformers
!pip install accelerate --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.5/157.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.1 MB/s eta 0:00:00


### Tokenizer

In [ ]:
from transformers import AutoTokenizer

# distilbert-base-cased
tokenizer1 = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")
print(tokenizer1)

# roberta-base
tokenizer2 = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
print(tokenizer2)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

RobertaTokenizerFast(name_or_path='FacebookAI/roberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)


### Models

In [ ]:
from transformers import AutoModelForSequenceClassification

# distilbert-base-cased
model1 = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=6)

# roberta-base
model2 = AutoModelForSequenceClassification.from_pretrained('FacebookAI/roberta-base', num_labels=6)

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Dataset

In [ ]:
from datasets import load_dataset, DatasetDict

ds = load_dataset("mteb/emotion")

def truncate(example):
    return {
        'text': " ".join(example['text'].split()),
        'label': example['label'],
        'label_text': example['label_text']
    }

small_ds = DatasetDict(
    train=ds['train'].shuffle(seed=24).select(range(128)).map(truncate),
    val=ds['validation'].shuffle(seed=24).select(range(32)).map(truncate),
    test=ds['test'].shuffle(seed=24).select(range(32)).map(truncate)
    )

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/128k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15956 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1988 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1986 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [ ]:
small_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 128
    })
    val: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 32
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 32
    })
})

In [ ]:
print(small_ds["train"][:10])

{'text': ['i cant help but feel how much burdened my parents are', 'id have to get to the class for eight dance for an hour nine get home ten if im lucky eat i cant eat before a class as dancing when full makes me feel vile sit around digesting etc ish then get to bed and try to sleep before getting up unnaturally early', 'i realise that although i originally started this blog for a specific purpose it has really grown beyond that and i shouldnt feel pressured to writing about specific things', 'i never thought id feel so much as a jot of sympathy for hussein whom i always viewed as a jumped up petty thug whatever my thoughts may be about actions against his administration', 'i fully understand the feeling of being beaten down wounded and bereft', 'i feel so stressed out with family problems', 'i just feel really violent right now', 'i feel like people like this arent getting caught therefore the government plays it up when they catch criminals of petty crimes to make themselves look b

### Training #1: distilbert-base-cased

From above:

```
tokenizer1 = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")

model1 = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=6)
```



In [ ]:
from transformers import DataCollatorWithPadding

def tokenize_function1(examples):
    return tokenizer1(examples["text"], padding=True, truncation=True)

small_tokenized_ds = small_ds.map(tokenize_function1, batched=True, batch_size=16)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer1)

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification

accuracy = evaluate.load("accuracy")

arguments1 = TrainingArguments(
    output_dir="sample_cl_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=8,
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

trainer1 = Trainer(
    model=model1,
    args=arguments1,
    train_dataset=small_tokenized_ds['train'],
    eval_dataset=small_tokenized_ds['val'],
    processing_class=tokenizer1,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer1.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.463800,1.524798,0.375000
2,1.365000,1.460427,0.375000
3,1.263300,1.396498,0.437500
4,1.179000,1.369205,0.437500
5,1.137800,1.361834,0.437500


TrainOutput(global_step=40, training_loss=1.2817914962768555, metrics={'train_runtime': 50.1989, 'train_samples_per_second': 12.749, 'train_steps_per_second': 0.797, 'total_flos': 9426555724608.0, 'train_loss': 1.2817914962768555, 'epoch': 5.0})

Evaluation

In [ ]:
fine_tuned_model1 = AutoModelForSequenceClassification.from_pretrained("sample_cl_trainer/checkpoint-40")

result1 = trainer1.evaluate(small_tokenized_ds['test'])
print(result1)

In [ ]:
import evaluate
from transformers import AutoModelForSequenceClassification
import torch

metric = evaluate.load("accuracy")

model_inputs = tokenizer1(list(small_tokenized_ds['test']['text']), padding=True, truncation=True, return_tensors='pt')
outputs = fine_tuned_model1(**model_inputs, output_hidden_states=True)
predictions = torch.argmax(outputs.logits, dim=-1)
accuracy = metric.compute(predictions=predictions, references=small_tokenized_ds['test']['label'])
print(accuracy)

Visualization

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import re
import torch
import tensorflow as tf
import tensorboard as tb

In [ ]:
fine_tuned_model1 = AutoModelForSequenceClassification.from_pretrained("sample_cl_trainer/checkpoint-40")

model_inputs = tokenizer1(list(small_tokenized_ds['val']['text']), padding=True, truncation=True, return_tensors='pt')
outputs = fine_tuned_model1(**model_inputs, output_hidden_states=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import os

path = "results_vis"
layer=0
if not os.path.exists(path):
  os.mkdir(path)

while layer in range(len(outputs['hidden_states'])):
  if not os.path.exists(path+'/layer_' + str(layer)):
    os.mkdir(path+'/layer_' + str(layer))

  example = 0
  tensors = []
  labels = []

  while example in range(len(outputs['hidden_states'][layer])):
    sp_token_position = 0
    for token in model_inputs['input_ids'][example]:
      if token != 101:
        sp_token_position += 1
      else:
        tensor = outputs['hidden_states'][layer][example][sp_token_position]
        tensors.append(tensor)
        break

    label = [small_tokenized_ds['test']['text'][example],str(small_tokenized_ds['test']['label'][example])]
    labels.append(label)
    example +=1

  writer=SummaryWriter(path+'/layer_' + str(layer))
  writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['Text','Emotion'])

  layer+=1

### Training #2: roberta-base

From above:


```
tokenizer2 = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")

model2 = AutoModelForSequenceClassification.from_pretrained('FacebookAI/roberta-base', num_labels=6)
```



In [ ]:
def tokenize_function2(examples):
    return tokenizer2(examples["text"], padding=True, truncation=True)

small_tokenized_ds = small_ds.map(tokenize_function2, batched=True, batch_size=16)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer2)

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [ ]:
arguments = TrainingArguments(
    output_dir="sample_cl_trainer2",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=8,
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

trainer2 = Trainer(
    model=model2,
    args=arguments,
    train_dataset=small_tokenized_ds['train'],
    eval_dataset=small_tokenized_ds['val'],
    processing_class=tokenizer1,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer2.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.530100,1.531916,0.312500
2,1.444000,1.540251,0.312500
3,1.425100,1.485779,0.375000
4,1.379900,1.463665,0.343750
5,1.363100,1.452500,0.406250


TrainOutput(global_step=40, training_loss=1.428449273109436, metrics={'train_runtime': 125.2069, 'train_samples_per_second': 5.112, 'train_steps_per_second': 0.319, 'total_flos': 17744189044608.0, 'train_loss': 1.428449273109436, 'epoch': 5.0})

Evaluation

In [ ]:
fine_tuned_model2 = AutoModelForSequenceClassification.from_pretrained("sample_cl_trainer2/checkpoint-40")

result2 = trainer2.evaluate(small_tokenized_ds['test'])
print(result2)

In [ ]:
metric = evaluate.load("accuracy")
fine_tuned_model2 = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/test/checkpoint-40")

model_inputs = tokenizer2(list(small_tokenized_ds['test']['text']), padding=True, truncation=True, return_tensors='pt')
outputs = fine_tuned_model2(**model_inputs, output_hidden_states=True)
predictions = torch.argmax(outputs.logits, dim=-1)
accuracy = metric.compute(predictions=predictions, references=small_tokenized_ds['test']['label'])
print(accuracy)

Visualization

In [ ]:
model_inputs = tokenizer2(list(small_tokenized_ds['val']['text']), padding=True, truncation=True, return_tensors='pt')
outputs = fine_tuned_model2(**model_inputs, output_hidden_states=True)

In [ ]:
path = "results_vis"
layer=0
if not os.path.exists(path):
  os.mkdir(path)

while layer in range(len(outputs['hidden_states'])):
  if not os.path.exists(path+'/layer_' + str(layer)):
    os.mkdir(path+'/layer_' + str(layer))

  example = 0
  tensors = []
  labels = []

  while example in range(len(outputs['hidden_states'][layer])):
    tensor = outputs['hidden_states'][layer][example][0]
    tensors.append(tensor)

    label = [small_tokenized_ds['test']['text'][example],str(small_tokenized_ds['test']['label'][example])]
    labels.append(label)
    example +=1

  writer=SummaryWriter(path+'/layer_' + str(layer))
  writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['Text','Emotion'])

  layer+=1